In [1]:
!pip install -e /home/sagar/modeling-info-spread

Obtaining file:///home/sagar/modeling-info-spread
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.3.2
    Uninstalling src-0.3.2:
      Successfully uninstalled src-0.3.2
  Running setup.py develop for src


In [2]:
import os
import matplotlib.pyplot as plt
from matplotlib import colormaps
from src.models.NSI import NSI
from src.models.channel import *
from tqdm.notebook import tqdm
import pickle
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from scipy import stats
from collections import defaultdict
import numpy as np
import networkx as nx

# Channels

7 Channel Topologies

1. Binary Symmetric
2. Binary Asymmetric
3. Triangle
4. Cherry
5. Paw Channel
6. BA Model

Differences between 1 & 2 show differences between symmetric and asymmetric error probabilities, differences between 3 & 4 show differences between unique situation where one uniform and other is heterogeneous degree distributions but both have same steady state, 5 & 6 show effect of heterogeneous degree distribution. 

For the binary asymmetric channel, we make the asymmetry $|\epsilon_1 - \epsilon_\alpha| = 0.4$, but also a subset where we vary this. $\epsilon_1$ is fixed as $0.4$.

All channels are one-flip only. 

In [43]:
## Errors

lin_errors = np.linspace(0,1,21)
log_errors = np.logspace(-10,0,11)

log_errors = log_errors[log_errors != 0.1] # removing 0.1 because that's a duplicate

In [44]:
lin_errors, log_errors

(array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
        0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
 array([1.e-10, 1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03,
        1.e-02, 1.e+00]))

In [5]:
## Binary Symmetric

bin_sym_channels = {
    
    'lin': {err: single_flip_channel(1, err) for err in lin_errors},
    'log': {err: single_flip_channel(1, err) for err in log_errors}
    
}

In [6]:
## Binary Asymmetric

bin_asym_channels = {
    
    'fixed': {
        
        'lin': {err: bin_asym_channel(err, (err - 0.4 )% 1) for err in lin_errors},
        'log': {err: bin_asym_channel(1, (err - 0.4) % 1) for err in log_errors}
        
    },
    
    'varying': {
        
        'lin': {err: bin_asym_channel(0.3, (0.3 - err) % 1) for err in lin_errors},
        'log': {err: bin_asym_channel(0.3, (0.3 - err) % 1) for err in log_errors}
        
    }
    
}

In [7]:
## Triangle

triangle_channels = {
    
    'lin': {err: simplex_channel(3, err) for err in lin_errors},
    'log': {err: simplex_channel(3, err) for err in log_errors}
    
}

In [52]:
def star_channel_new(n: int,
                     e: float) -> np.ndarray:

    """
    Generate a star-shaped channel (one node with n-1 degree) where the hub node is always message 0.
    Error of departing is the same for hub and spokes.

    :param n: number of nodes/messages
    :param e: error (probability of departing from original message)

    :return: Numpy array of transition probabilities
    """

    err = e/(n-1)

    hub_probs: np.ndarray = np.array([1-e] + [err]*(n-1))
    spoke_probs: list = list()

    for i in range(1,n):
        i_probs: np.ndarray = np.array([err] + [0]*(n-1))
        i_probs[i] = 1 - e
        spoke_probs.append(i_probs)

    all_probs = [hub_probs] + spoke_probs
    Q: np.ndarray = np.array(all_probs)

    return Q

In [53]:
## Cherry 

cherry_channels = {
    
    'lin': {err: star_channel_new(3, err) for err in lin_errors},
    'log': {err: star_channel_new(3, err) for err in log_errors}
    
}

In [9]:
## Paw Channel 

paw_channels = {
    
    'lin': {err: paw_channel(err) for err in lin_errors},
    'log': {err: paw_channel(err) for err in log_errors}
    
}

In [12]:
## BA Channel

G = nx.barabasi_albert_graph(n=100, m=2)

M = nx.adjacency_matrix(G).todense()

BA_channels = {
    
    'lin': {err: symmetric_graphical_channel(M, err) for err in lin_errors},
    'log': {err: symmetric_graphical_channel(M, err) for err in log_errors}
    
}

# Constants

In [22]:
N = 10000
T = 1000

k = 4
beta = 0.1

num_sims = 20

# Simulations

In [46]:
dirpath = "/data_users1/sagar/modeling-info-spread"

In [47]:
all_channels = [bin_sym_channels, bin_asym_channels, triangle_channels, 
                cherry_channels, paw_channels, BA_channels] 
all_names = ['bin_sym', 'bin_asym', 'triangle', 'cherry', 'paw', 'BA']

In [55]:
trunc_all_channels = all_channels[3:]
trunc_all_names = all_names[3:]

In [56]:
# Running all simulations with seed 0 

for name, channel_type in zip(trunc_all_names, trunc_all_channels): # trunc because cherry did
    
    print(f"Now simulating dynamics on {name} channels.")
    
    directory = dirpath + '/' + name
    
    try:
        os.mkdir(directory)
        
    except FileExistsError:
        pass
    
    directory = directory + '/seed0/'
    
    try:
        os.mkdir(directory)
        
    except FileExistsError:
        pass
    
    if name != 'bin_asym':
        
        for scale, channel_dict in channel_type.items():
            
            for err, channel in channel_dict.items():
                
                model = NSI(N=N, T=T, P=channel)
                
                for n in range(num_sims):
                    
                    subsubdir = directory + '/' + str(err)
                    
                    try:
                        os.mkdir(subsubdir)

                    except FileExistsError:
                        pass
                    
                    model.homogeneous_simulation(beta=beta, k=k, filepath=f"{subsubdir}/{n}", pbar_on=False)
                
    else:
        
        for key, error_type in channel_type.items():
            
            subdir = directory +"/" + key
            
            try:
                os.mkdir(subdir)

            except FileExistsError:
                pass
        
            for scale, channel_dict in error_type.items():

                for err, channel in channel_dict.items():
                    model = NSI(N=N, T=T, P=channel)
                    
                    for n in range(num_sims):
                    
                        subsubdir = subdir + '/' + str(err) 

                        try:
                            os.mkdir(subsubdir)

                        except FileExistsError:
                            pass
                    
                        model.homogeneous_simulation(beta=beta, k=k, filepath=f"{subsubdir}/{n}", pbar_on=False)

Now simulating dynamics on cherry channels.


TypeError: object of type 'numpy.float64' has no len()

In [63]:
range(cherry_channels['lin'][0.0].shape[0])

range(0, 3)

In [ ]:
# Seeding all other messages

for name, channel_type in zip(all_names,all_channels):
    
    print(f"Now simulating dynamics on {name} channels.")
    
    directory = dirpath + '/' + name
    
    if name != 'bin_asym':
        
        for scale, channel_dict in channel_type.items():
            
            for err, channel in channel_dict.items():
                
                alphabet_size = channel.shape[0]
                
                model = NSI(N=N, T=T, P=channel)
                
                for message in range(len(alphabet_size[1:])):
                    
                    subdir = directory + '/' + f"seed{message}"
                        
                    try:
                        os.mkdir(subdir)

                    except FileExistsError:
                        pass
                    
                    seed_dist = np.zeroes(channel.shape[0])
                    seed_dist[message] = 1
                
                    for n in range(num_sims):

                        subsubdir = subdir + '/' + str(err)

                        try:
                            os.mkdir(subsubdir)

                        except FileExistsError:
                            pass

                        model.homogeneous_simulation(beta=beta, k=k, filepath=f"{subsubdir}/{n}", pbar_on=False,
                                                    seedI=seed_dist)
                
    else:
        
        for key, error_type in channel_type.items():
            
            subdir = directory +"/" + key
            
            try:
                os.mkdir(subdir)

            except FileExistsError:
                pass
        
            for scale, channel_dict in error_type.items():

                for err, channel in channel_dict.items():
                    
                    model = NSI(N=N, T=T, P=channel)
                    alphabet_size = channel.shape[0]
                    
                    for message in range(len(alphabet_size)): # running asym on all messages due to error last time
                    
                        subsubdir = directory + '/' + f"seed{message}"

                        try:
                            os.mkdir(subsubdir)

                        except FileExistsError:
                            pass
                        
                        seed_dist = np.zeroes(channel.shape[0])
                        seed_dist[message] = 1

                        for n in range(num_sims):

                            subsubsubdir = subsubdir + '/' + str(err)

                            try:
                                os.mkdir(subsubsubdir)

                            except FileExistsError:
                                pass

                            model.homogeneous_simulation(beta=beta, k=k, filepath=f"{subsubdir}/{n}", pbar_on=False,
                                                    seed_=seed_dist)

# Calculating Numerical Results